# wps_split_merged_climos

#### wps_split_merged_climos is a process that runs the [split_merged_climos](https://github.com/pacificclimate/climate-explorer-data-prep#split_merged_climos-split-climo-means-files-into-per-interval-files-month-season-year) module of PCIC Climate Explorer Data Preparation Tools. Here, the client will try to connect to a remote osprey instance using the url parameter.

In [1]:
from birdy import WPSClient
from pkg_resources import resource_filename
import requests
import os
from nchelpers import CFDataset
from bs4 import BeautifulSoup

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "thunderbird":
    os.chdir('../')

url = 'https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thunderbird/wps'
thunderbird = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [2]:
# NBVAL_IGNORE_OUTPUT
thunderbird.split_merged_climos?

Signature: thunderbird.split_merged_climos(netcdf=None, loglevel='INFO')
Docstring:
Split climo means files into one file per time interval

Parameters
----------
netcdf : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    NetCDF files to process
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
output : ComplexData:mimetype:`application/metalink+xml; version=4.0`
    Metalink object between output files
File:      ~/Desktop/thunderbird/</home/sangwonl/Desktop/thunderbird/venv/lib/python3.6/site-packages/birdy/client/base.py-3>
Type:      method


#### We can use the docstring to ensure we provide the appropriate parameters.

In [3]:
# Test local and opendap files
tasmax_climos_local = resource_filename('tests', 'data/tiny_downscaled_tasmax_climos.nc')
hydromodel_climos_opendap = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/tiny_hydromodel_gcm_climos.nc"

output = thunderbird.split_merged_climos([tasmax_climos_local, hydromodel_climos_opendap])

In [4]:
req = requests.get(output.get()[0])
split_filepaths = [out_file.contents[0].strip() for out_file in BeautifulSoup(BeautifulSoup(req.content.decode('utf-8')).prettify()).find_all('metaurl')]

#### Once the process has completed we can extract the results and ensure it is what we expected.

In [5]:
# Helper function copied from test_split_merged_climos.py of ce-dataprep
# https://github.com/pacificclimate/climate-explorer-data-prep/blob/master/tests/test_split_merged_climos.py#L13

def basename_components(filepath):
    """Returns the CMOR(ish) components of the basename (filename) of the given filepath."""
    # Slightly tricky because variable names can contain underscores, which separate components.
    # We find the location f of the frequency component in the split and use it to assemble the components properly.
    pieces = os.path.basename(filepath).split('_')
    f = next(i for i, piece in enumerate(pieces) if 'Clim' in piece)
    return ['_'.join(pieces[:f])] + pieces[f:]

In [6]:
# Check output filenames

assert {basename_components(fp)[0] for fp in split_filepaths} == {'+'.join(sorted(CFDataset(input_file).dependent_varnames())) for input_file in [tasmax_climos_local, hydromodel_climos_opendap]}
assert {basename_components(fp)[1] for fp in split_filepaths} == {'mClim', 'sClim', 'aClim'}